In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2

In [3]:
html_data = requests.get("https://www.otomoto.pl/oferta/bmw-m5-600km-m5-individual-kolor-carbon-pakiet-ceramiczne-hamulce-salon-pl-ID6DQdad.html")
soup = BeautifulSoup(html_data.content, 'html5lib')

In [4]:
eur = 4.57
print(eur)


4.57


In [5]:
def get_data(link):
    price = soup.find('span',attrs={"class":"offer-price__number"}).text
    price = price.replace(" ","")
    if "PLN" in price:
        price = price.replace("PLN","")
        price = float(price)
    elif "EUR" in price:
        price = price.replace("EUR","")
        price = float(price)
        price = price * eur
    else:
        price = None
    print(price)

In [6]:
price = soup.find('span',attrs={"class":"offer-price__number"}).text
price = price.replace(" ","")
if "PLN" in price:
    price = price.replace("PLN","")
    price = float(price)
elif "EUR" in price:
    price = price.replace("EUR","")
    price = float(price)
    price = price * eur
else:
    price = None
print(price)


649900.0


In [7]:
ID = soup.find('div',attrs={"class":"offer-content__metabar"})
ID = ID.find('div')
ID = ID.find('span',attrs={"id":"ad_id"}).text
print(ID)

6085517757


In [8]:
ID = soup.find('div',attrs={"class":"offer-content__metabar"})

In [9]:
def take_details(soup):
    details = {}
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip()
        value = info.find('div').text.strip()
        details[key] = value
    return details

In [10]:
details = take_details(soup)
details

{'Oferta od': 'Firmy',
 'Kategoria': 'Osobowe',
 'Marka pojazdu': 'BMW',
 'Model pojazdu': 'M5',
 'Rok produkcji': '2021',
 'Przebieg': '10 000 km',
 'Pojemność skokowa': '4 395 cm3',
 'Rodzaj paliwa': 'Benzyna',
 'Moc': '600 KM',
 'Skrzynia biegów': 'Automatyczna',
 'Napęd': '4x4 (dołączany automatycznie)',
 'Typ nadwozia': 'Sedan',
 'Liczba drzwi': '4',
 'Liczba miejsc': '5',
 'Kolor': 'Inny kolor',
 'Faktura VAT': 'Tak',
 'Kraj pochodzenia': 'Polska',
 'Pierwsza rejestracja': '16/05/2021',
 'Pierwszy właściciel': 'Tak',
 'Bezwypadkowy': 'Tak',
 'Serwisowany w ASO': 'Tak',
 'Stan': 'Używane'}

In [11]:
def data_clean(details):
    try:
        details['Przebieg'] = details['Przebieg'].replace(" km","").replace(" ","")
    except:
        pass
    try:
        details['Moc'] = details['Moc'].replace(" KM","")
    except:
        pass
    try:
        details['Pojemność skokowa'] = details['Pojemność skokowa'].replace(" cm3","").replace(" ","")
    except:
        pass
    #details['lub do (przebieg km)'] = details['lub do (przebieg km)'].replace("km","").replace(" ","")
    return details

In [12]:
details

{'Oferta od': 'Firmy',
 'Kategoria': 'Osobowe',
 'Marka pojazdu': 'BMW',
 'Model pojazdu': 'M5',
 'Rok produkcji': '2021',
 'Przebieg': '10 000 km',
 'Pojemność skokowa': '4 395 cm3',
 'Rodzaj paliwa': 'Benzyna',
 'Moc': '600 KM',
 'Skrzynia biegów': 'Automatyczna',
 'Napęd': '4x4 (dołączany automatycznie)',
 'Typ nadwozia': 'Sedan',
 'Liczba drzwi': '4',
 'Liczba miejsc': '5',
 'Kolor': 'Inny kolor',
 'Faktura VAT': 'Tak',
 'Kraj pochodzenia': 'Polska',
 'Pierwsza rejestracja': '16/05/2021',
 'Pierwszy właściciel': 'Tak',
 'Bezwypadkowy': 'Tak',
 'Serwisowany w ASO': 'Tak',
 'Stan': 'Używane'}

In [13]:
# def geturl(url):
#     html_data_url = requests.get(url)
#     soup_url = BeautifulSoup(html_data_url.content, 'html5lib')
#     location = soup.find('a',attrs={"class":"css-1ibwe9h e1nbpvi62"}).text
#     new_row = {}
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#             new_row = offer['href']

In [14]:
html_data_url = requests.get("https://www.otomoto.pl/osobowe/")
soup_url = BeautifulSoup(html_data_url.content, 'html5lib')

# new_row = {}
# for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}).find_all('a',attrs={"class":"offer-title__link"}):
#     new_row = get_data(offer['href'])

In [15]:
# links = []
# new_row = {}
# for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#     if "https://www.otomoto.pl" in offer['href']:
#         links.append(offer['href'])
# print(links)

In [16]:
# all_data = []
# for page in links:
#     html_data = requests.get(page)
#     soup = BeautifulSoup(html_data.content, 'html5lib')
#     details = take_details(soup)
#     details = data_clean(details)
#     all_data.append(details)
# print(all_data)

In [17]:
# pd.DataFrame.from_dict(all_data, orient='columns')

In [23]:
Read_announcements = 0
links = []
new_row = {}
for page_main in range(1,3):  
    try:
        html_data_link = requests.get(f"https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?page={page_main}")
        page_main += 1
    except Exception:
        print("Link_ERROR")
        continue
    for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
        if "https://www.otomoto.pl" in offer['href']:
            links.append(offer['href'])
        Read_announcements += 1
        clear_output(wait=True)
        print("Read_announcements ", Read_announcements)
        all_data = []
        for page in links:
            html_data = requests.get(page)
            soup = BeautifulSoup(html_data.content, 'html5lib')
            details = take_details(soup)
            details = data_clean(details)
            all_data.append(details)
pd.DataFrame.from_dict(all_data, orient='columns')

Read_announcements  72


,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Wersja,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,...,Serwisowany w ASO,Filtr cząstek stałych,Leasing,Opłata początkowa,Miesięczna rata,Liczba pozostałych rat,Wartość wykupu,VAT marża,Możliwość finansowania,Gwarancja dealerska (w cenie)
0,Osoby prywatnej,Osobowe,Mercedes-Benz,Klasa E,200 T CDI DPF BlueEFFICIENCY Automatik,W212 (2009-2016),2011,332000,2143,Diesel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Osoby prywatnej,Osobowe,Ford,Focus,1.8 TDCi FX Gold,Mk2 (2004-2011),2005,274125,1753,Diesel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Osoby prywatnej,Osobowe,Audi,A6,Avant 2.0 TDI Ultra S tronic,C7 (2011-2018),2018,43000,1968,Diesel,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Osoby prywatnej,Osobowe,Mercedes-Benz,GLA,250 4-Matic,NaN,2015,64275,1991,Benzyna,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Osoby prywatnej,Osobowe,Audi,Q5,3.0 TDI (clean diesel) quattro S tronic,8R (2008-2016),2015,164000,2967,Diesel,...,Tak,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Firmy,Osobowe,Ford,Focus,NaN,NaN,2018,120349,1499,Diesel,...,Tak,NaN,Tak,NaN,NaN,NaN,NaN,NaN,Tak,44 miesięcy
60,Firmy,Osobowe,Nissan,Leaf,NaN,NaN,2017,47510,NaN,Elektryczny,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,Tak,Tak,37 miesięcy
61,Firmy,Osobowe,Opel,Adam,NaN,NaN,2014,98317,1398,Benzyna+LPG,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,Tak,Tak,12 miesięcy
62,Firmy,Osobowe,Nissan,Qashqai,NaN,II (2013-),2016,129358,1598,Diesel,...,Tak,NaN,Tak,NaN,NaN,NaN,NaN,NaN,Tak,26 miesięcy


In [19]:
get_data("https://www.otomoto.pl/oferta/ferrari-f8-tributo-official-ferrari-dealer-ID6E2r6Z.html")

43900.0


In [20]:
print(new_row)

{}


In [21]:
Link_ERROR = 0
Read_announcements = 0
# page = 1
# while True:
    
for page in range(1,101):  
    try:
        html_data_link = requests.get(f"https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?page={page}")
        page += 1
    except Exception:
        print("Link_ERROR")
        continue
    
    soup_link = BeautifulSoup(html_data_link.content, 'html5lib')
    
    if soup_link.find('div',attrs={"class":"css-pi2gf8 e1qwpsp45"}) is None:
        pass
    else:
        break

    for offer in soup_link.find_all('a',attrs={"data-cy":"listing-item-link"}):
            new_row = get_data("https://www.otodom.pl"+ offer['href'])
            
            
            if new_row == None:
                Link_ERROR += 1
            else:
                df_table = df_table.append(new_row, ignore_index=True)
                Read_announcements += 1
                clear_output(wait=True)
                print("Read_announcements ", Read_announcements)
                print("Link_ERROR ",Link_ERROR)     
                
print("Finish")

43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0


43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0


43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
43900.0
